In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2
scikit-learn-intelex==2021.20210714.120553


In [23]:
import pickle
import pandas as pd
import numpy as np
import os

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [3]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [9]:
np.mean(y_pred)

16.191691679979066

In [12]:
year = 2021
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [27]:
df_result = pd.DataFrame({'ride_id': df['ride_id'].values, 'predictions': y_pred})

In [28]:
output_dir = f'output/{year:04d}/{month:02d}'
os.makedirs(output_dir, exist_ok=True)

In [29]:
output_file = f'{output_dir}/predictions.parquet'

In [30]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [33]:
!du -hcs output/2021/02/predictions.parquet

19M	output/2021/02/predictions.parquet
19M	total


In [34]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1493 bytes to starter.py
